In [ ]:
import pyspark.sql
import pyspark.sql.functions as sf

In [ ]:
spark = pyspark.sql.SparkSession.Builder().getOrCreate()

# 1 Fictional Sales Data

In this example we use a fictional data set of company revenues. The special property of this data set is that a company can have a different company as its parent company. Eventually a business expert wants to see the whole revenue of a company including all child companies. This requires that we build up an additional table containing all children (direct and indirect) for every company, such that we can join the revenues against this table and then aggregate over all direct and indirect children for each parent.

Let's start by loading and inspecting the data.

In [ ]:
basedir = "s3://dimajix-training/data"

In [ ]:
data = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(basedir + "/global-sales.csv")

data.printSchema()

In [ ]:
data.toPandas()

# 2 Single Step of transitive parent-child relations

In the next step we want to build the helper table containing all children for every company. We will calculate this table using an iterative algorithm which adds the next level of children in every iteration. We first implement a single iteration, which will add the next level of children to each parent company.

In [ ]:
# Remove all records without a parent company for the algorithm
cleaned_df = data \
    .filter(data["parent_company"].isNotNull()) \
    .select(data["company"], data["parent_company"])

In [ ]:
def iterate_parent_child(df):
    # Denote the incoming table "parent" and "child", since we will do a self-join and the join condition would be ambigious without aliases otherwise
    parent_df = df.alias("parent")
    child_df = df.alias("child")
    
    # Calculate next levels of indirect children by joining the table to itself and by retrieving the child of each child of each parent
    next_level = # YOUR CODE HERE
    
    # Add current relations, otherwise they will be lost
    cur_level = # YOUR CODE HERE
    
    # Return union of next indirection and current relations
    return next_level.union(cur_level).distinct()

### Perform one iteration

Now let us perform a single iteration and inspect the result.

In [ ]:
next = # YOUR CODE HERE

next.orderBy("parent_company","company").toPandas()

# 3 Iterative Algorithm

Now that we can add one level of indirection to our table of parent-child relations, we simply need to apply this algorithm as often as new records are created. We also add a reflective relation of each company to itself at the end, such that when using the table for aggregating all children, the revenue of each company itself  will also be added up in addition to its children.

In [ ]:
def calc_transitive_children(df):
    # Remove records without a parent
    cleaned_df = data \
        .filter(data["parent_company"].isNotNull()) \
        .select(data["company"], data["parent_company"])
    
    # Iterate as long as new records are created
    cur_df = cleaned_df
    cur_count = cur_df.count()

    while (True):
        # Perform next iteration using iterate_parent_child
        next_df = # YOUR CODE HERE
        next_count = next_df.count()
        
        # If no new records are created, we are finished
        # YOUR CODE HERE
        
        # This would be a good place to perform a checkpoint
        cur_df = next_df        
        cur_count = next_count
        
    # Create additional reflective relation of each company to irself
    self_df = data.select(sf.col("company").alias("parent_company"), sf.col("company"))
    
    return self_df.union(cur_df).distinct()

### Run Algorithm

Now let us run the whole algorithm on the original data set and inspect the result.

In [ ]:
relations = # YOUR CODE HERE

relations.orderBy("parent_company","company").toPandas()

### Inspect execution plan

In [ ]:
# YOUR CODE HERE

# 4 Perform Aggregation

Now let us perform the final aggregation, such that we can calculate the revenue of each company including each direct and indirect child. This can be performed by joining the `relations` data frame to the original `data` data frame and then grouping on the `parent_company` column of the `relations` data frame and adding up the revenue.

In [ ]:
hierarchical_revenue = # YOUR CODE HERE

In [ ]:
hierarchical_revenue.toPandas()

### Check Totals

Just to verify the result, let us compare the result of company 1 ("Earth") with a simple sum over all revenues.

In [ ]:
totals = data.select(sf.sum(data["revenue"]))
totals.toPandas()